In [1]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV

In [2]:
path = './open'
list_name = 'Stock_List.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [3]:
# select start, end data

start_date = '20200613'
end_date = '20211031'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

WEEKDAY of "start_date" : 5
NUM of WEEKS to "end_date" : 43
HOW MANY "Business_days" : (360, 1)


,Date
0,2020-06-15
1,2020-06-16
2,2020-06-17
3,2020-06-18
4,2020-06-19


In [4]:
sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()
print(Business_days)
# print(sample)

sample = pd.merge(Business_days, sample, how = 'outer')
sample=sample.drop(['Date'],axis=1)
sample.Close = sample.Close.ffill()
sample.Close = sample.Close.bfill()
sample=sample.to_numpy()
sample=sample.reshape(72,5)
print((sample))
# break
# sample['weekday'] = sample.Date.apply(lambda x : x.weekday())
# sample['weeknum'] = sample.Date.apply(lambda x : x.strftime('%V'))
# sample.Close = sample.Close.ffill()
# sample = pd.pivot_table(data = sample, values = 'Close', columns = 'weekday', index = 'weeknum')
# print(sample)
# break
# sample.head(70)

          Date
0   2020-06-15
1   2020-06-16
2   2020-06-17
3   2020-06-18
4   2020-06-19
..         ...
355 2021-10-25
356 2021-10-26
357 2021-10-27
358 2021-10-28
359 2021-10-29

[360 rows x 1 columns]
[[49900. 52100. 52200. 52300. 52900.]
 [52000. 51400. 52900. 51900. 53300.]
 [52400. 52800. 52600. 52900. 53600.]
 [55000. 53400. 53000. 52800. 52700.]
 [53400. 53800. 54700. 53800. 54400.]
 [54200. 55300. 54700. 54100. 54200.]
 [55600. 58600. 59000. 59000. 57900.]
 [56800. 57300. 56900. 58000. 57500.]
 [57800. 58200. 59000. 58700. 58000.]
 [58000. 58400. 57800. 55400. 55900.]
 [56100. 56400. 56400. 55600. 55400.]
 [54000. 54200. 54400. 56400. 55600.]
 [56500. 58700. 58400. 59200. 59000.]
 [60400. 61000. 61000. 59500. 59300.]
 [59200. 58200. 58600. 57800. 57900.]
 [58200. 58200. 58200. 58200. 58200.]
 [58700. 59000. 59900. 59700. 59700.]
 [60400. 60900. 60900. 60000. 59500.]
 [60000. 60900. 60900. 60100. 60200.]
 [60400. 59800. 59000. 58100. 56600.]
 [57400. 58800. 58500. 60300. 60100.

In [5]:
# training, predict method
def get_prediction(x_close, y_close, x_close_public):
    
    prediction_close = 0.0
    for i, model in enumerate(models):         
        model.fit(x_close, y_close)
        prediction_close += model.predict(np.expand_dims(x_close_public, 0))*models_rate[i]
    
    return prediction_close

sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path,sample_name))

In [6]:
model1 = LinearRegression()
# model2 = RandomForestRegressor(criterion="mae", n_estimators=200)
models = [model1]


model1_rate = 1
# model2_rate = 0.3
models_rate = [model1_rate]

for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')

    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()
    data_close=data.drop(['Date'],axis=1)

    data['ma5']=data['Close'].ewm(5).mean()
    data['ma20']=data['Close'].ewm(20).mean()
    data_ma5=data.drop(['Date','Close','ma20'],axis=1)
    data_ma20=data.drop(['Date','Close','ma5'],axis=1)


    data_close=data_close.to_numpy()
    data_close=data_close.reshape(72,5)

    data_ma5=data_ma5.to_numpy()
    data_ma5=data_ma5.reshape(72,5)

    data_ma20=data_ma20.to_numpy()
    data_ma20=data_ma20.reshape(72,5)

    # data['weekday'] = data.Date.apply(lambda x : x.weekday())
    # data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    # data.Close = data.Close.ffill()
    # data.Close = data.Close.bfill()





    # data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    # data_ma5 = pd.pivot_table(data = data, values = 'ma5', columns = 'weekday', index = 'weeknum')
    # data_ma20 = pd.pivot_table(data = data, values = 'ma20', columns = 'weekday', index = 'weeknum')

    # data flatten  # data flatten for reshape
    data_close_flatten = data_close
    data_ma5_flatten = data_ma5
    data_ma20_flatten = data_ma20
    predictions = []
    predictions_ma5 = []
    predictions_ma20 = []

    
    # predict day 1

    # close
    data_close_day1 = data_close_flatten.reshape((-1,1))
    data_ma5_day1 = data_ma5_flatten.reshape((-1,1))
    data_ma20_day1 = data_ma20_flatten.reshape((-1,1))

    x_close = data_close_day1[:-2]  # training x
    y_close = data_close_day1[1:-1]  # training y
    x_close_public = data_close_day1[-1]  # predict x
    print(x_close_public)


    x_ma5 = data_ma5_day1[:-2]  # training x
    y_ma5 = data_ma5_day1[1:-1]  # training y
    x_ma5_public = data_ma5_day1[-1]  # predict x

    x_ma20 = data_ma20_day1[:-2]  # training x
    y_ma20 = data_ma20_day1[1:-1]  # training y
    x_ma20_public = data_ma20_day1[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close)
    
    # predict day 1 finish
    
    
    # predict day 2
    
    # close

    data_close_day2 = data_close_flatten.reshape((-1,2))
    data_ma5_day2 = data_ma5_flatten.reshape((-1,2))
    data_ma20_day2 = data_ma20_flatten.reshape((-1,2))


    x_close = data_close_day2[:-2]
    y_close = data_close_day2[1:-1][:, 1]  # 2일차만 훈련, 예측함
    x_close_public = data_close_day2[-1]   

    x_ma5 = data_ma5_day2[:-2]  # training x
    y_ma5 = data_ma5_day2[1:-1][:,1]  # training y
    x_ma5_public = data_ma5_day2[-1]  # predict x

    x_ma20 = data_ma20_day2[:-2]  # training x
    y_ma20 = data_ma20_day2[1:-1][:,1]  # training y
    x_ma20_public = data_ma20_day2[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close)
    
    # predict day 2 finish
    
    
    # predict day 3
    
    # close
    # data_close_day3 = np.insert(data_close_day3, 0, data_close_day3[0])
    # data_close_day3 = data_close_day3.reshape((-1, 3))

    # data_ma5_day3 = np.insert(data_ma5_day3, 0, data_ma5_day3[0])
    # data_ma5_day3 = data_ma5_day3.reshape((-1, 3))

    # data_ma20_day3 = np.insert(data_ma20_day3, 0, data_ma20_day3[0])
    # data_ma20_day3 = data_ma20_day3.reshape((-1, 3))

    data_close_day3 = data_close_flatten.reshape((-1,3))
    data_ma5_day3 = data_ma5_flatten.reshape((-1,3))
    data_ma20_day3 = data_ma20_flatten.reshape((-1,3))

    x_close = data_close_day3[:-2]
    y_close = data_close_day3[1:-1][:, 2]
    x_close_public = data_close_day3[-1]

    x_ma5 = data_ma5_day3[:-2]  # training x
    y_ma5 = data_ma5_day3[1:-1][:,2]  # training y
    x_ma5_public = data_ma5_day3[-1]  # predict x

    x_ma20 = data_ma20_day3[:-2]  # training x
    y_ma20 = data_ma20_day3[1:-1][:,2]  # training y
    x_ma20_public = data_ma20_day3[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close)
    # predict day 3 finish

    
    # predict day 4
    
    # close
    # data_close_day4 = np.insert(data_close_flatten, 0, data_close_flatten[0])
    # data_close_day4 = np.insert(data_close_day4, 0, data_close_day4[0])
    # data_close_day4 = np.append(data_close_day4, 0)
    # data_close_day4 = np.append(data_close_day4, 0)
    # data_close_day4 = np.append(data_close_day4, 0)
    data_close_day4 = data_close_flatten.reshape((-1, 4))

    # data_ma5_day4 = np.insert(data_ma5_flatten, 0, data_ma5_flatten[0])
    # data_ma5_day4 = np.insert(data_ma5_day4, 0, data_ma5_day4[0])
    # data_ma5_day4 = np.append(data_ma5_day4, 0)
    # data_ma5_day4 = np.append(data_ma5_day4, 0)
    # data_ma5_day4 = np.append(data_ma5_day4, 0)
    data_ma5_day4 = data_ma5_flatten.reshape((-1, 4))

    # data_ma20_day4 = np.insert(data_ma20_flatten, 0, data_ma20_flatten[0])
    # data_ma20_day4 = np.insert(data_ma20_day4, 0, data_ma20_day4[0])
    # data_ma20_day4 = np.append(data_ma20_day4, 0)
    # data_ma20_day4 = np.append(data_ma20_day4, 0)
    # data_ma20_day4 = np.append(data_ma20_day4, 0)
    data_ma20_day4 = data_ma20_flatten.reshape((-1, 4))

    x_close = data_close_day4[:-2]
    y_close = data_close_day4[1:-1][:, 3]
    x_close_public = data_close_day4[-1]

    x_ma5 = data_ma5_day4[:-2]  # training x
    y_ma5 = data_ma5_day4[1:-1][:,3]  # training y
    x_ma5_public = data_ma5_day4[-1]  # predict x

    x_ma20 = data_ma20_day4[:-2]  # training x
    y_ma20 = data_ma20_day4[1:-1][:,3]  # training y
    x_ma20_public = data_ma20_day4[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close)
    
    # predict day 4 finish
    
    # predict day 5
    
    # close
    data_close_day5 = data_close_flatten.reshape((-1, 5))
    data_ma5_day5 = data_ma5_flatten.reshape((-1, 5))
    data_ma20_day5 = data_ma20_flatten.reshape((-1, 5))
    
    x_close = data_close_day5[:-2]
    y_close = data_close_day5[1:-1][:, 4]
    x_close_public = data_close_day5[-1]

    x_ma5 = data_ma5_day5[:-2]  # training x
    y_ma5 = data_ma5_day5[1:-1][:,4]  # training y
    x_ma5_public = data_ma5_day5[-1]  # predict x

    x_ma20 = data_ma20_day5[:-2]  # training x
    y_ma20 = data_ma20_day5[1:-1][:,4]  # training y
    x_ma20_public = data_ma20_day5[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close)
    # predict day 5 finish
    print(type(predictions))
    #sample_submission.loc[:,code] = predictions*2
    sample_submission.loc[:4,code] = predictions
sample_submission.isna().sum().sum()

  0%|          | 0/370 [00:00<?, ?it/s]C:\Users\user\.conda\envs\stockpredict\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
  0%|          | 1/370 [00:00<01:42,  3.61it/s]

[69800.]
<class 'list'>


C:\Users\user\.conda\envs\stockpredict\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
  1%|          | 2/370 [00:00<01:35,  3.87it/s]

[103000.]
<class 'list'>


C:\Users\user\.conda\envs\stockpredict\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
  1%|          | 3/370 [00:00<01:25,  4.32it/s]

...

C:\Users\user\.conda\envs\stockpredict\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
 10%|█         | 38/370 [00:08<01:20,  4.11it/s]

[537000.]
<class 'list'>


C:\Users\user\.conda\envs\stockpredict\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
 11%|█         | 39/370 [00:08<01:23,  3.99it/s]C:\Users\user\.conda\envs\stockpredict\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
 11%|█         | 40/370 [00:08<01:12,  4.52it/s]

[103000.]
<class 'list'>
[94600.]
<class 'list'>


KeyboardInterrupt: 

In [ ]:

start_date = '20211101'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

real=[]
real_data=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\stock_predict2\\open\\sample_submission.csv')
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()

    real_data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    real_data_close_flatten = np.ravel(real_data_close.to_numpy())
    # print(real_data_close_flatten)
    

    real_data.loc[:4,code]=real_data_close_flatten

type(real_data)
type(sample_submission)

label_np = real_data.to_numpy()[0:5,1:]
pred_np = sample_submission.to_numpy()[0:5, 1:]
print(pred_np)


result=np.mean(abs(label_np-pred_np)/label_np)*100
print(result)

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 44
HOW MANY "Business_days" : (5, 1)


,Date
0,2021-11-01
1,2021-11-02
2,2021-11-03
3,2021-11-04
4,2021-11-05


In [ ]:
real=[]
real_data=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\stock_predict2\\open\\sample_submission.csv')
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()

    real_data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    real_data_close_flatten = np.ravel(real_data_close.to_numpy())
    # print(real_data_close_flatten)
    

    real_data.loc[:4,code]=real_data_close_flatten


real_data

100%|██████████| 370/370 [01:22<00:00,  4.48it/s]


,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,27850,35200,60000,146000,103000,16600,49750,84300,106500,...,48300,51700,80500,36750,25700,52000,348500,261600,26600,17600
1,2021-11-02,29250,35050,61700,148500,107000,17350,48950,86000,107500,...,50800,54100,81600,35850,25950,52900,348000,258600,27100,18100
2,2021-11-03,30250,34050,61500,145500,110000,17100,50300,85000,105500,...,49450,54600,82700,34700,25450,51200,346500,253700,26750,18300
3,2021-11-04,29450,33800,61100,145000,125500,17750,50600,87000,106000,...,48700,53800,81500,35500,25500,50600,356500,249600,26350,17550
4,2021-11-05,29550,33450,60600,144500,133500,17300,50400,88000,107000,...,49800,53900,85100,35500,25100,50200,383500,247800,26050,17550
5,2021-11-29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2021-11-30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2021-12-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2021-12-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2021-12-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


[[27752.46973379302 34820.608978367636 60028.24789887636 ...
  261875.13345381085 27147.533067890745 17262.14910380305]
 [27986.23874409854 34861.56370477938 60370.69474880529 ...
  262652.75999312435 27168.758416794593 17481.15078948463]
 [28465.614556007014 34861.20372251644 60212.01554397037 ...
  262438.85518538585 27186.545079233914 17870.385439314283]
 [29724.159426561408 34542.20770512955 60471.50422704749 ...
  262802.1647145297 27286.44848997733 18019.904200855224]
 [28421.669131754734 35037.26516017033 59651.994932836315 ...
  259969.1932021677 27079.822508249297 18239.820821829606]]
3.1050343555302744


In [ ]:
#private
# select start, end data

start_date = '20200711'
end_date = '20211126'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

WEEKDAY of "start_date" : 5
NUM of WEEKS to "end_date" : 47
HOW MANY "Business_days" : (360, 1)


,Date
0,2020-07-13
1,2020-07-14
2,2020-07-15
3,2020-07-16
4,2020-07-17


In [ ]:
model1 = LinearRegression()
# model2 = RandomForestRegressor(criterion="mae", n_estimators=200)
models = [model1]


model1_rate = 1
# model2_rate = 0.3
models_rate = [model1_rate]

for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')

    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()
    data_close=data.drop(['Date'],axis=1)

    data['ma5']=data['Close'].ewm(5).mean()
    data['ma20']=data['Close'].ewm(20).mean()
    data_ma5=data.drop(['Date','Close','ma20'],axis=1)
    data_ma20=data.drop(['Date','Close','ma5'],axis=1)


    data_close=data_close.to_numpy()
    data_close=data_close.reshape(72,5)

    data_ma5=data_ma5.to_numpy()
    data_ma5=data_ma5.reshape(72,5)

    data_ma20=data_ma20.to_numpy()
    data_ma20=data_ma20.reshape(72,5)

    # data['weekday'] = data.Date.apply(lambda x : x.weekday())
    # data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    # data.Close = data.Close.ffill()
    # data.Close = data.Close.bfill()





    # data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    # data_ma5 = pd.pivot_table(data = data, values = 'ma5', columns = 'weekday', index = 'weeknum')
    # data_ma20 = pd.pivot_table(data = data, values = 'ma20', columns = 'weekday', index = 'weeknum')

    # data flatten  # data flatten for reshape
    data_close_flatten = data_close
    data_ma5_flatten = data_ma5
    data_ma20_flatten = data_ma20
    predictions = []
    predictions_ma5 = []
    predictions_ma20 = []

    
    # predict day 1

    # close
    data_close_day1 = data_close_flatten.reshape((-1,1))
    data_ma5_day1 = data_ma5_flatten.reshape((-1,1))
    data_ma20_day1 = data_ma20_flatten.reshape((-1,1))

    x_close = data_close_day1[:-2]  # training x
    y_close = data_close_day1[1:-1]  # training y
    x_close_public = data_close_day1[-1]  # predict x

    x_ma5 = data_ma5_day1[:-2]  # training x
    y_ma5 = data_ma5_day1[1:-1]  # training y
    x_ma5_public = data_ma5_day1[-1]  # predict x

    x_ma20 = data_ma20_day1[:-2]  # training x
    y_ma20 = data_ma20_day1[1:-1]  # training y
    x_ma20_public = data_ma20_day1[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close)
    
    # predict day 1 finish
    
    
    # predict day 2
    
    # close

    data_close_day2 = data_close_flatten.reshape((-1,2))
    data_ma5_day2 = data_ma5_flatten.reshape((-1,2))
    data_ma20_day2 = data_ma20_flatten.reshape((-1,2))


    x_close = data_close_day2[:-2]
    y_close = data_close_day2[1:-1][:, 1]  # 2일차만 훈련, 예측함
    x_close_public = data_close_day2[-1]   

    x_ma5 = data_ma5_day2[:-2]  # training x
    y_ma5 = data_ma5_day2[1:-1][:,1]  # training y
    x_ma5_public = data_ma5_day2[-1]  # predict x

    x_ma20 = data_ma20_day2[:-2]  # training x
    y_ma20 = data_ma20_day2[1:-1][:,1]  # training y
    x_ma20_public = data_ma20_day2[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close)
    
    # predict day 2 finish
    
    
    # predict day 3
    
    # close
    # data_close_day3 = np.insert(data_close_day3, 0, data_close_day3[0])
    # data_close_day3 = data_close_day3.reshape((-1, 3))

    # data_ma5_day3 = np.insert(data_ma5_day3, 0, data_ma5_day3[0])
    # data_ma5_day3 = data_ma5_day3.reshape((-1, 3))

    # data_ma20_day3 = np.insert(data_ma20_day3, 0, data_ma20_day3[0])
    # data_ma20_day3 = data_ma20_day3.reshape((-1, 3))

    data_close_day3 = data_close_flatten.reshape((-1,3))
    data_ma5_day3 = data_ma5_flatten.reshape((-1,3))
    data_ma20_day3 = data_ma20_flatten.reshape((-1,3))

    x_close = data_close_day3[:-2]
    y_close = data_close_day3[1:-1][:, 2]
    x_close_public = data_close_day3[-1]

    x_ma5 = data_ma5_day3[:-2]  # training x
    y_ma5 = data_ma5_day3[1:-1][:,2]  # training y
    x_ma5_public = data_ma5_day3[-1]  # predict x

    x_ma20 = data_ma20_day3[:-2]  # training x
    y_ma20 = data_ma20_day3[1:-1][:,2]  # training y
    x_ma20_public = data_ma20_day3[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close)
    # predict day 3 finish

    
    # predict day 4
    
    # close
    # data_close_day4 = np.insert(data_close_flatten, 0, data_close_flatten[0])
    # data_close_day4 = np.insert(data_close_day4, 0, data_close_day4[0])
    # data_close_day4 = np.append(data_close_day4, 0)
    # data_close_day4 = np.append(data_close_day4, 0)
    # data_close_day4 = np.append(data_close_day4, 0)
    data_close_day4 = data_close_flatten.reshape((-1, 4))

    # data_ma5_day4 = np.insert(data_ma5_flatten, 0, data_ma5_flatten[0])
    # data_ma5_day4 = np.insert(data_ma5_day4, 0, data_ma5_day4[0])
    # data_ma5_day4 = np.append(data_ma5_day4, 0)
    # data_ma5_day4 = np.append(data_ma5_day4, 0)
    # data_ma5_day4 = np.append(data_ma5_day4, 0)
    data_ma5_day4 = data_ma5_flatten.reshape((-1, 4))

    # data_ma20_day4 = np.insert(data_ma20_flatten, 0, data_ma20_flatten[0])
    # data_ma20_day4 = np.insert(data_ma20_day4, 0, data_ma20_day4[0])
    # data_ma20_day4 = np.append(data_ma20_day4, 0)
    # data_ma20_day4 = np.append(data_ma20_day4, 0)
    # data_ma20_day4 = np.append(data_ma20_day4, 0)
    data_ma20_day4 = data_ma20_flatten.reshape((-1, 4))

    x_close = data_close_day4[:-2]
    y_close = data_close_day4[1:-1][:, 3]
    x_close_public = data_close_day4[-1]

    x_ma5 = data_ma5_day4[:-2]  # training x
    y_ma5 = data_ma5_day4[1:-1][:,3]  # training y
    x_ma5_public = data_ma5_day4[-1]  # predict x

    x_ma20 = data_ma20_day4[:-2]  # training x
    y_ma20 = data_ma20_day4[1:-1][:,3]  # training y
    x_ma20_public = data_ma20_day4[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close)
    
    # predict day 4 finish
    
    # predict day 5
    
    # close
    data_close_day5 = data_close_flatten.reshape((-1, 5))
    data_ma5_day5 = data_ma5_flatten.reshape((-1, 5))
    data_ma20_day5 = data_ma20_flatten.reshape((-1, 5))
    
    x_close = data_close_day5[:-2]
    y_close = data_close_day5[1:-1][:, 4]
    x_close_public = data_close_day5[-1]

    x_ma5 = data_ma5_day5[:-2]  # training x
    y_ma5 = data_ma5_day5[1:-1][:,4]  # training y
    x_ma5_public = data_ma5_day5[-1]  # predict x

    x_ma20 = data_ma20_day5[:-2]  # training x
    y_ma20 = data_ma20_day5[1:-1][:,4]  # training y
    x_ma20_public = data_ma20_day5[-1]  # predict x

    prediction_close = get_prediction(x_close, y_close, x_close_public)
    prediction_ma5 = get_prediction(x_ma5, y_ma5, x_ma5_public)
    prediction_ma20 = get_prediction(x_ma20, y_ma20, x_ma20_public)
    predictions.append(prediction_close)
    # predict day 5 finish
        
    #sample_submission.loc[:,code] = predictions*2
    sample_submission.loc[5:9,code] = predictions
sample_submission.isna().sum().sum()


  0%|          | 0/370 [00:00<?, ?it/s]C:\Users\user\.conda\envs\stockpredict\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
  0%|          | 1/370 [00:00<01:33,  3.95it/s]C:\Users\user\.conda\envs\stockpredict\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
  1%|          | 2/370 [00:00<01:27,  4.21it/s]C:\Users\user\.conda\envs\stockpredict\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecation

0

In [ ]:
sample_submission.to_csv('Final.csv',index=False)
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,27752.469734,34820.608978,60028.247899,142975.381395,103045.631062,16723.179573,47394.725048,85109.633940,103121.059299,...,48961.527599,52026.314610,82402.870838,35940.641945,26234.273167,53157.670877,334640.241965,261875.133454,27147.533068,17262.149104
1,2021-11-02,27986.238744,34861.563705,60370.694749,144118.164375,103344.548237,16747.692419,48065.141844,85029.614746,103098.497136,...,48788.899565,52005.602055,82481.650724,35724.762599,26551.025118,52844.285144,334624.346865,262652.759993,27168.758417,17481.150789
2,2021-11-03,28465.614556,34861.203723,60212.015544,145755.592461,103483.091416,17025.090548,47467.638735,85036.953939,102335.668008,...,48209.536750,51453.261598,82840.650571,35274.531049,27034.113343,53486.435357,332671.699366,262438.855185,27186.545079,17870.385439
3,2021-11-04,29724.159427,34542.207705,60471.504227,147165.122425,104846.770682,16787.480936,49033.520557,85735.542400,103304.056628,...,48753.187153,50776.922036,83294.195706,36019.848601,27372.847407,53069.982528,335478.176258,262802.164715,27286.448490,18019.904201
4,2021-11-05,28421.669132,35037.265160,59651.994933,146788.072218,107233.888626,17141.965899,46736.746462,85297.857004,101595.266164,...,49874.405076,49300.461388,83875.088991,34193.639127,26611.644255,54164.018303,326668.370117,259969.193202,27079.822508,18239.820822
5,2021-11-29,31862.270364,31085.010022,59527.836629,130089.683775,120660.764563,15568.047226,44860.864014,80632.075258,115540.183672,...,44556.377561,48766.747655,102511.039066,39090.187395,20347.259269,50672.808291,372288.010828,276616.277534,25037.850242,16898.139609
6,2021-11-30,31833.684347,31263.660996,59773.616643,131142.882612,120770.645139,15724.476522,45486.184139,80526.024158,115590.018037,...,44558.057231,48767.570769,104029.808375,38946.624115,20731.551276,50464.696727,369383.675918,278204.437915,25094.833756,17212.877080
7,2021-12-01,32089.973588,31462.368994,60008.598437,132016.931300,121793.849904,15779.026325,46100.977423,81125.058228,116224.382597,...,45159.102008,48604.166469,103788.935575,39096.005010,20751.429692,50271.250596,372580.068846,275373.724358,25123.990246,17733.110468
8,2021-12-02,31739.509395,31461.801992,60253.726158,132416.048968,124153.471158,15906.598553,46676.494956,81999.574781,115081.759523,...,44595.834519,48735.749248,105209.375661,37503.369208,21415.751892,50568.195016,369595.238513,277321.848052,25163.085231,17559.677804
9,2021-12-03,31976.919363,31836.976507,60650.988546,133953.003599,123732.517126,15966.094333,46510.678010,83971.279836,116048.394297,...,44942.331895,48791.136810,105846.592718,39047.858877,20101.655822,50798.668579,370731.770235,276929.771739,25569.043449,18403.670291
